In [41]:
import nltk
print('hello world')

hello world


In [42]:
def foo():
    print('foo')

In [43]:
foo()

foo


In [44]:
import pandas
#pandas.read_excel(r'C:\Users\n1555085\Downloads\Copy of CSAT Help hub April Responses.xlsx')
sheet = pandas.read_excel(r'C:\Users\n1555085\Downloads\May Help Hub Data.xlsx')
courtesy = sheet['Unnamed: 9'].dropna().values.tolist()[1:]
effectiveness = sheet['Unnamed: 10'].dropna().values.tolist()[1:]
timeliness = sheet['Unnamed: 11'].dropna().values.tolist()[1:]
understanding = sheet['Unnamed: 12'].dropna().values.tolist()[1:]
nps = sheet['Unnamed: 13'].dropna().values.tolist()[1:]
comments = courtesy + effectiveness + timeliness + understanding + nps
date = sheet['Unnamed: 1'].dropna().values.tolist()[1:]
#should add completion rate

In [45]:
print(f'# of courtesy comments: {len(courtesy)}')
print(f'# of effectiveness comments: {len(effectiveness)}')
print(f'# of timeliness comments: {len(timeliness)}')
print(f'# of understanding comments: {len(understanding)}')
print(f'# of nps comments: {len(nps)}')
print(f'# of comments: {len(comments)}')

# of courtesy comments: 10
# of effectiveness comments: 17
# of timeliness comments: 24
# of understanding comments: 13
# of nps comments: 148
# of comments: 212


In [46]:
print('hi')

hi


In [47]:
#read from corpus, remove stopwords
from nltk.corpus import stopwords
from string import punctuation

with open(r'C:\Users\n1555085\Downloads\Project\positiveComments.txt', 'r') as f:
    posReviews = f.readlines()
with open(r'C:\Users\n1555085\Downloads\Project\negativeComments.txt', 'r') as f:
    negReviews = f.readlines()
print(len(posReviews), len(negReviews))

sw = set(stopwords.words('english') + list(punctuation))
notStopwords = ['not', 'no', '!', 'but', 'too', 'have', 'had']

def removeStopwords(review):
    return ' '.join([word for word in review.split() if word not in sw or word in notStopwords])
posReviews = list(filter(lambda s: s , list(map(removeStopwords, posReviews))))
negReviews = list(filter(lambda s: s , list(map(removeStopwords, negReviews))))

posReviews[0:5]
#negReviews[0:5]

1954 888


['Spenser Cameron set assisted IT issues. He exceptional trying assist newcomer new systems.',
 'second consultant awesome appropriate follow well letting know break fix consultants suggestion since I could not receive emails. She best.',
 'support personnel helpful patient issue I',
 'fast service.',
 'Greeting robotic, however loosened conversation went on.']

In [48]:
#define pos and neg bag-of words, and vocabulary

posWords = [word.lower() for review in posReviews for word in review.split()]
negWords = [word.lower() for review in negReviews for word in review.split()]
vocabulary = list(set(posWords + negWords))
#vocabulary[0:100]

In [49]:
#define training data: list of (review list of words, label) tuples
trainingData = [(r.split(), 'pos') for r in posReviews] + [(r.split(), 'neg') for r in negReviews]

In [50]:
#NAIVE BAYES CLASSIFIER: extract feature vector from each review, train classifier
def featureVector(review):
    reviewWords = set(review)
    features = {}
    for word in vocabulary:
        features[word] = word in reviewWords
    return features

naiveBayesClassifier = nltk.NaiveBayesClassifier.train(nltk.classify.apply_features(featureVector, trainingData))

In [51]:
#classify functions
def NBclassify(review):
    review = removeStopwords(review)
    features = featureVector(review.split())
    probDist = naiveBayesClassifier.prob_classify(features)
    confidence = max(probDist.prob('pos'), probDist.prob('neg'))
    #pos and neg add to 1
    return (naiveBayesClassifier.classify(features).upper(), confidence)

def NBclassifyComments(comments):
    return [(c, NBclassify(c)) for c in comments]

In [52]:
labeledComments = NBclassifyComments(effectiveness)
labeledComments

[('1st ticket they marked resolved even though it was not.',
  ('POS', 0.9366672276167465)),
 ('no next steps. have to restart my request from scratch.',
  ('POS', 0.618661262748057)),
 ('See previous comment.', ('POS', 0.9999648658586715)),
 ('No response nor assistance was given', ('POS', 0.9996889424613481)),
 ('I was given a solution that did not meet my needs. Finally I was directed to a JIRA site for another team and my ticket was resolved although my issue still remains. Hopefully the JIRA team can help.',
  ('NEG', 0.9967503117422805)),
 ('there is a fix without calling internet which i was told to do',
  ('POS', 0.9878065638447872)),
 ('.', ('POS', 0.9998960483520787)),
 ('I was asked multiple times to keep doing the same thing by different people. Clear my cache and cookies when I literally had just done that 1 minute prior. That was a bit frustrating.',
  ('NEG', 0.9999167705781233)),
 ('Unfortunately recreating my VDE is not an ideal scenario. Having said that, it has nothi

In [36]:
print(stopwords.words('english'))
# note, might want to remove following stopwords: ['not', 'no', '!', 'but', 'too', 'have', 'had']

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '